In [12]:
# Usual imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (18, 10)

# OpenCV
import cv2

# Keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical

# Adversarial Robustness Toolbox
from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent

# Other modules
from pathlib import Path
from glob import glob

# ART complains if we don't use this
import tensorflow
tensorflow.compat.v1.disable_eager_execution()

Similar to what we did in Project 1, we will now define some constant for the path to annotations and images

In [5]:
DATA_FOLDER = Path('../data')
IMG_FOLDER = Path(DATA_FOLDER / 'tsrd-train')
ANNOTATIONS = Path(DATA_FOLDER / 'TsignRecgTrain4170Annotation.txt')
COL_NAMES = ['filename','width','height','x1','y1','x2','y2','label']
MODEL_FOLDER = Path('../Part 1')
MODEL = Path(MODEL_FOLDER / 'chinese_traffic_sign_classifier_v1.h5')

In [6]:
df = pd.read_csv(ANNOTATIONS, names=COL_NAMES, sep=';', header=None, index_col=False)

We can now load the model we created in Project 1

In [7]:
classifier = load_model(MODEL)

2021-10-07 16:39:46.326929: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2021-10-07 16:39:46.327018: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: CPU00370U
2021-10-07 16:39:46.327037: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: CPU00370U
2021-10-07 16:39:46.327214: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 460.91.3
2021-10-07 16:39:46.327280: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 460.91.3
2021-10-07 16:39:46.327296: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 460.91.3
2021-10-07 16:39:46.392331: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical 

The ART library uses its own format for a model, so we need to create a new one that takes our model as an input. Following hints for additional parameters

In [8]:
classifier_art = KerasClassifier(model=classifier, clip_values=(0.0, 1.0), use_logits=False)

In order to create the adversarial examples, we will first import all the images. We use the same function calls from Project 1:

In [9]:
images = glob(str(IMG_FOLDER) + '/*.png')
IMG_SIZE = (134,128)

dataset = []
labels = []

for idx in range(len(images)):
    full_img_path = images[idx]
    filename = Path(full_img_path).name
    img_bgr = cv2.imread(full_img_path)
    image = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, IMG_SIZE)
    image = img_to_array(image)
    dataset.append(image)
    # get label specific to this image
    row = df[df['filename'] == filename]
    label = row['label'].values[0]
    if row.empty:
        print(filename)
    else:
        labels.append(label)

dataset = np.array(dataset) / 255.0
labels = to_categorical(labels)

Before saving the model as we did in Project 1, we will create the split between training and test data.

In [14]:
(train_X, valid_X, train_Y, valid_Y) = train_test_split(dataset, labels,
                                                        test_size=0.2, stratify=labels)

np.savez('../data/chinese_traffic_sign_dataset_2.npz', train_X=train_X, valid_X=valid_X,
         train_Y=train_Y, valid_Y=valid_Y)

Ok, we are now ready to generate malicious examples from the training data. We will use the two methods presented in the milestone description, and we will compare them with the original data points as well as with examples corrupted with random uniform noise.